In [0]:
from pyspark.sql.functions import concat, col, lit
import mlflow
from yaml import safe_load

with open('../params.yml', 'r') as f:
      params = safe_load(f)

CATALOG = params.get('data_params')['catalog']
SCHEMA = params.get('data_params')['schema']



In [0]:
inc_status = (spark.read.option("readChangeData", "true")
      .option('startingVersion', '11')
      .table(f"{CATALOG}.{SCHEMA}.device_status"))

inc_status.createOrReplaceTempView("inc_status_view")

In [0]:
triage_devices = spark.sql("""
    WITH tablestatuserrors AS (
        SELECT *, 
               ai_query('databricks-claude-sonnet-4', 
                        CONCAT('what is a telemetry error for the following mobile tower device? Please be extremely concise and give only the example telemetry error, preferably 1 sentence max.', device)
               ) AS telemetry_error 
        FROM inc_status_view 
        WHERE status = 'Telemetry Issue'
    )
    SELECT tower_id, device_id, device, status, event_timestamp, telemetry_error
    FROM tablestatuserrors
""")
triage_devices.write.mode('append').insertInto(f"{CATALOG}.{SCHEMA}.device_status_triage")